In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qutip import *
from qiskit import *

In [ ]:
# Constants

I = np.eye(2)
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])

In [ ]:
# "Check" helper functions

def check_trace(rho):
    """
    Check if the trace of the density matrix is 1.
    """
    return np.isclose(rho.tr(), 1.0, atol=1e-10)

def check_hermitian(H):
    """
    Check if the operator is Hermitian.
    """
    return np.allclose(H, H.dag(), atol=1e-10)

def check_unitary(U):
    """
    Check if the operator is unitary.
    """
    return np.allclose(U * U.dag(), I, atol=1e-10) and np.allclose(U.dag() * U, I, atol=1e-10)

def check_psd(mat):
    """
    Check if the matrix is positive semi-definite.
    """
    return np.all(np.linalg.eigvals(mat) >= -1e-10)

def check_choi_decomposition(choi_matrix, pos_choi, neg_choi):
    """
    Check if the Choi matrix is decomposed correctly.
    """
    return np.all(choi_matrix - pos_choi + neg_choi >= -1e-10)


In [ ]:
# "Compute" helper functions

def compute_fidelity(rho1, rho2):
    """
    Calculate the fidelity between two density matrices.
    """
    return (rho1 * rho2).tr().sqrt()

def compute_expectation_value(rho, A):
    """
    Calculate the expectation value of an operator A with respect to a density matrix rho.
    """
    return (rho * A).tr()

In [ ]:
def choi_adjointchoi(choi_matrix):
    """
    Convert the Choi matrix to its adjoint form, and vice versa (As defined in the supplementary material of the paper).
    """
    U_23 = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
    adjoint_choi = np.conjugate(np.dot(U_23,np.dot(choi_matrix,U_23)))
    
    return adjoint_choi

def construct_choi_matrix(map):

    # TODO: verify if this is the correct way to construct the Choi matrix, currently generated by copilot 


    """
    Construct the Choi matrix for the map.
    """
    # dim = map.shape[0]
    # choi_matrix = np.zeros((dim**2, dim**2), dtype=complex)
    
    # for i in range(dim):
    #     for j in range(dim):
    #         for k in range(dim):
    #             for l in range(dim):
    #                 choi_matrix[i*dim + j, k*dim + l] = map[i, k] * np.conjugate(map[j, l])
    w = np.array([[np.array([1,0,0,1]), np.array([0,1,1j,0])], [np.array([0,1,-1j,0]), np.array([1,0,0,-1])]])

    choi_matrix = np.zeros((4,4), dtype=complex)
    for i in range(2):
        for j in range(2):
            Tw = np.dot(map, w[i,j])
            eval_Eij = Tw[0]*I + Tw[1]*sigma_x + Tw[2]*sigma_y + Tw[3]*sigma_z
            choi_matrix[2*i:2*i+2,2*j:2*j+2] = eval_Eij/2
    
    return choi_matrix


def positive_and_negative_eigenspaces(choi_matrix):
    """
    Compute the positive and negative eigenspaces of the Choi matrix.
    """
    eigenvalues, eigenvectors = np.linalg.eig(choi_matrix)
    
    # Positive eigenspace
    pos_eigenvalues = eigenvalues[eigenvalues > 0]
    pos_eigenvectors = eigenvectors[:, eigenvalues > 0]
    
    # Negative eigenspace
    neg_eigenvalues = eigenvalues[eigenvalues < 0]
    neg_eigenvectors = eigenvectors[:, eigenvalues < 0]
    
    return pos_eigenvalues, pos_eigenvectors, neg_eigenvalues, neg_eigenvectors


def construct_CP_map_from_eigenspace(eigenvalues, eigenvectors):
    """
    Construct the map from the positive and negative eigenspaces.
    """
    dim = eigenvectors.shape[0]
    CP_map = np.zeros((dim, dim), dtype=complex)
    
    for i in range(len(eigenvalues)):
        CP_map += eigenvalues[i] * np.outer(eigenvectors[:, i], eigenvectors[:, i].conjugate())
    
    return CP_map


def decompose_into_extremal_CP_maps(choi_matrix_adjoint):

    # choi_matrix_cp_map_adjoint = choi_matrix_cp_map.T

    A = choi_matrix_adjoint[:2, :2]
    C = choi_matrix_adjoint[:2, 2:]
    C_dag = choi_matrix_adjoint[2:, :2]
    B = choi_matrix_adjoint[2:, 2:]

    # assert C.conjugate().T == C_dag, "C and C_dag are not conjugate transpose"
    # assert I - A  == B, "B is not equal to I - A"

    R = np.linalg.inv(np.sqrt(A)) @ C @ np.linalg.inv(np.sqrt(B))

    V, S, W_dag = np.linalg.svd(R)
    S = np.diag(S)

    theta1 = np.arccos(S[0, 0])
    theta2 = np.arccos(S[1, 1])

    U1_diag = np.diag([np.exp(1j * theta1), np.exp(1j * theta2)])
    U2_diag = np.diag([np.exp(-1j * theta1), np.exp(-1j * theta2)])

    U1 = (1/2)*(V @ U1_diag @ W_dag)
    U2 = (1/2)*(V @ U2_diag @ W_dag) 
    
    choi1 = np.zeros((4, 4), dtype=complex)
    choi2 = np.zeros((4, 4), dtype=complex)

    choi1[:2, :2] = A
    choi1[:2, 2:] = np.sqrt(A) @ U1 @ np.sqrt(B)
    choi1[2:, :2] = np.sqrt(B) @ U1.conjugate().T @ np.sqrt(A)
    choi1[2:, 2:] =  B

    choi2[:2, :2] = A
    choi2[:2, 2:] = np.sqrt(A) @ U2 @ np.sqrt(B)
    choi2[2:, :2] = np.sqrt(B) @ U2.conjugate().T @ np.sqrt(A)
    choi2[2:, 2:] = B
    
    return choi1, choi2


def choi_to_map_in_kraus_rep(choi_matrix):
    
    pass

def get_unitary_angles_from_choi(choi_extremal_adjoint):

    pass

In [ ]:
# Lindbladian to General Dynamical Map

def eval_lindbladian_operator_basis(H, l_ops, t_list):
    # Compute the result of Lindblad map acting on nice operator basis (I,X,Y,Z). Use mesolve with eigenstates as initial state.
    eig_xp = Qobj(np.array([1,1])/np.sqrt(2))
    eig_xm = Qobj(np.array([1,-1])/np.sqrt(2))
    eig_yp = Qobj(np.array([1,1j])/np.sqrt(2))
    eig_ym = Qobj(np.array([1,-1j])/np.sqrt(2))
    eig_zp = Qobj(np.array([1,0]))
    eig_zm = Qobj(np.array([0,1]))

    L_Id = np.array(mesolve(H,eig_xp,t_list, l_ops).states) + np.array(mesolve(H,eig_xm,t_list, l_ops).states)
    L_sigmax = np.array(mesolve(H,eig_xp,t_list, l_ops).states) - np.array(mesolve(H,eig_xm,t_list, l_ops).states)
    L_sigmay = np.array(mesolve(H,eig_yp,t_list, l_ops).states) - np.array(mesolve(H,eig_ym,t_list, l_ops).states)
    L_sigmaz = np.array(mesolve(H,eig_zp,t_list, l_ops).states) - np.array(mesolve(H,eig_zm,t_list, l_ops).states)
    return L_Id, L_sigmax, L_sigmay, L_sigmaz

def get_lindblad_matrix_from_map(L_Id, L_sigmax, L_sigmay, L_sigmaz):
    """
    Get the matrix form of the Lindland Map from the Hamiltonian and Lindblad operators. Reference: Equation (8.1.7) in OQS-book (Notes for PHYS 550).
    """
    L_Id_t = Qobj(L_Id)
    L_sigmax_t = Qobj(L_sigmax)
    L_sigmay_t = Qobj(L_sigmay)
    L_sigmaz_t = Qobj(L_sigmaz)  

    # Construct the Matrix with Equation (8.1.7) in OQS-book
    L_mat = (1/2)*np.array([
        [L_Id_t.tr(), L_sigmax_t.tr(), L_sigmay_t.tr(), L_sigmaz_t.tr()],
        [(sigmax()*L_Id_t).tr(), (sigmax()*L_sigmax_t).tr(), (sigmax()*L_sigmay_t).tr(), (sigmax()*L_sigmaz_t).tr()],
        [(sigmay()*L_Id_t).tr(), (sigmay()*L_sigmax_t).tr(), (sigmay()*L_sigmay_t).tr(), (sigmay()*L_sigmaz_t).tr()],
        [(sigmaz()*L_Id_t).tr(), (sigmaz()*L_sigmax_t).tr(), (sigmaz()*L_sigmay_t).tr(), (sigmaz()*L_sigmaz_t).tr()]    
    ])
    return L_mat

# def get_liouvillian_superoperator(H, l_ops):

#     # TODO: verify if this is the correct way to construct the Liouvillian superoperator, currently generated by copilot 


#     """
#     Get the Liouvillian superoperator from the Hamiltonian and Lindblad operators.
#     """
#     # Construct the Liouvillian superoperator
#     L = -1j * (tensor(H, qeye(2)) - tensor(qeye(2), H)).tr()
    
#     for l_op in l_ops:
#         L += (tensor(l_op.dag(), qeye(2)) @ tensor(qeye(2), l_op) - 0.5 * (tensor(l_op.dag() * l_op, qeye(2)) + tensor(qeye(2), l_op.dag() * l_op))).tr()
    
#     return L


# def get_dynamical_map(L, dt):
#     """
#     Get the dynamical map from the Liouvillian superoperator for time dt.
#     """
#     # Time evolution operator
#     U = (-1j * L * dt).expm()  
    
#     return U



In [ ]:
gamma = 1
beta = 1
omega = 1
times = np.linspace(0,1,101)
times = times[20:]
H = -sigmaz()/2
l_ops = []
l_ops.append([lindblad_dissipator(sigmam(),sigmam()),gamma*np.exp(beta*omega)])
l_ops.append([lindblad_dissipator(sigmap(),sigmap()),gamma])

t_idx = 5
L_Id, L_sigmax, L_sigmay, L_sigmaz = eval_lindbladian_operator_basis(H, l_ops, times)
L_matrix = get_lindblad_matrix_from_map(L_Id[t_idx], L_sigmax[t_idx], L_sigmay[t_idx], L_sigmaz[t_idx])
choi_L = construct_choi_matrix(L_matrix)
eigenspace_pm = positive_and_negative_eigenspaces(choi_L)

# Check if the Choi matrix is positive semi-definite. If postive, then the map is CP and can be decomposed into extremal maps.
if eigenspace_pm[3].size == 0:
    choi_adjoint = choi_adjointchoi(choi_L)
    [choi_extremal_1, choi_extremal_2] = decompose_into_extremal_CP_maps(choi_adjoint)
    
else:
    choi_plus = construct_CP_map_from_eigenspace(eigenspace_pm[0], eigenspace_pm[1])
    choi_minus = construct_CP_map_from_eigenspace(eigenspace_pm[2], eigenspace_pm[3])
    choi_plus_adjoint = choi_adjointchoi(choi_plus)
    choi_minus_adjoint = choi_adjointchoi(choi_minus)